In [1]:
# Envs
from dataset import MyData
from torchvision import transforms
from torch.utils.data import random_split, DataLoader
from train import *
from path import ROOT_DIR
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os

In [2]:
# load Data
csv_file = os.path.join(ROOT_DIR, "data", "labels", "car_imgs_4000.csv")
data = pd.read_csv(csv_file)

image_paths = data['filename'].tolist()
labels = data[['perspective_score_hood', 'perspective_score_backdoor_left']].values.tolist() 

In [3]:
# config 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 10
batch_size = 16
learning_rate = 0.0001
# image preporcessing
transform = transforms.Compose([
    transforms.Resize((448, 448)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
dataset = MyData(image_paths, labels, transform)

# load model
import torchvision.models as models
model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, 2), 
    nn.Sigmoid()  
)
model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_size = int(0.8 * len(dataset)) 
val_size = len(dataset) - train_size  
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss = validate(model, val_loader, criterion, device)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

torch.save(model.state_dict(), os.path.join(ROOT_DIR, "model", 'densenet121_model_10_1e-4_32_448x448.pth'))

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fdd0a82f700>
Traceback (most recent call last):
  File "/home/shaoxiang/anaconda3/envs/openmmlab/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    def __del__(self):
KeyboardInterrupt: 


OutOfMemoryError: CUDA out of memory. Tried to allocate 92.00 MiB. GPU 0 has a total capacty of 8.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 14.32 GiB is allocated by PyTorch, and 123.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
# test

test_image_path = os.path.join(ROOT_DIR, "data", "imgs", "00b9e159-3ce0-4325-8e9b-4d829db719e8.jpg")

In [8]:
from PIL import Image
def load_image(image_path):
    image = Image.open(image_path).convert('RGB') 
    image = transform(image) 
    image = image.unsqueeze(0) 
    return image

model = models.densenet121() 
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, 2),
    nn.Sigmoid() 
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.load_state_dict(torch.load(os.path.join(ROOT_DIR, "model", 'densenet121_model_10_1e-4_32_224x224.pth')))
model.eval()

image = load_image(test_image_path).to(device)
with torch.no_grad(): 
    output = model(image)  

output

tensor([[0.0088, 0.8496]], device='cuda:0')